In [1]:
# include("../src/BoundaryConditions.jl")
# include("../src/Configurations.jl")
# include("../src/Ensembles.jl")
# using .Ensembles

using ParallelTemperingMonteCarlo
using StaticArrays

In [2]:
x = NVT(10)

NVT(10)

In [3]:
function testtype(testinput::NVT)
    println("true")
end

function testmove(xinput::X) where X <: MoveType
    println("true")
end

testmove (generic function with 1 method)

In [4]:
testtype(x)

true


In [5]:
xmove = atommove()

typeof(xmove)

atommove

In [6]:
testmove(xmove)

true


In [8]:
bc = SphericalBC(radius=2.0)
v1 = SVector(1., 2., 3.)
conf = Config{3}([v1,v1,v1],bc)

envars_nvt = set_ensemble_variables(conf,NVT(1))
xx = typeof(envars_nvt)

NVTVariables

In [9]:
typeof(envars_nvt.index)

Int64

In [10]:
length(envars_nvt.trial_move) 

3

In [16]:
envars_nvt.trial_move = atom_displacement(SVector(0.,0.,0.),0.5,bc)

3-element SVector{3, Float64} with indices SOneTo(3):
  0.06797443479555543
 -0.15170255100268498
 -0.24777240290674468

In [17]:
envars_nvt.trial_move = atom_displacement(xmove,0.5,bc)

3-element SVector{3, Float64} with indices SOneTo(3):
 -0.3169025133439916
  0.5828375346844659
 -0.2222299593885947

In [ ]:
function metropolis_condition(delta_energy, beta)
    prob_val = exp(-delta_energy*beta)
    T = typeof(prob_val)
    return ifelse(prob_val > 1, T(1), prob_val)
end
function metropolis_condition(ensemble, delta_energy,volume_changed,volume_unchanged,beta)
    delta_h = d_en + ensemble.pressure*(volume_changed-volume_unchanged)
    prob_val = exp(-delta_h*beta + ensemble.n_atoms*log(volume_changed/volume_unchanged))
    T = typeof(prob_val)
    return ifelse(prob_val > 1, T(1), prob_val)
    return ifelse(prob_val > 1, T(1), prob_val)
end

function metropolis_condition(::atommove,mc_state,ensemble)
    return metropolis_condition((mc_state.new_en - mc_state.en_tot),mc_state.beta)
end
function metropolis_condition(::volumemove,mc_state,ensemble)
    return metropolis_condition(ensemble,(mc_state.new_en - mc_state.en_tot),mc_state.ensemble_variables.trial_config.bc.box_length^3,mc_states.config.bc.box_length^3,mc_state.beta )
end

In [ ]:
function acc_test!(mc_state,ensemble,movetype)
    if metropolis_condition(movetype,mc_state,ensemble) >=rand()
        swap_config!(mc_state,movetype)
    end
end


In [15]:
function mc_move!(mc_state,move_strat,pot,ensemble)
    #moveindex = rand(1:length(move_strat))
    mc_state.ensemble_variables.index = rand(1:length(move_strat))

    mc_state = generate_move!(mc_state,move_strat.movestrat[mc_state.ensemble_variables.index])

    mc_state = get_energy!(mc_state,pot,ensemble,move_strat.movestrat[mc_state.ensemble_variables.index])

    acc_test!(mc_state,ensemble,movetype)
    return mc_state
end
function mc_step!(mc_states,move_strat,pot,ensemble)
    for state in mc_states
        state = mc_move!(state,move_strat,pot,ensemble)
    end
    return mc_states
end
function mc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,n_steps,index)
    for i_step in 1:n_steps 
        mc_states=  mc_step!(mc_states,move_strat,pot,ensemble)
    end
    if rand() < 0.1
        parallel_tempering_exchange!(mc_states,mc_params,ensemble)
    end
    if rem(index,mc_params.n_adjust) == 0
        for state in mc_states 
            update_max_stepsize!(state,mc_params.n_adjust,ensemble)
        end
    end


    return mc_states
end
function mc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,n_steps,results,idx)

    mc_states = mc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,n_steps,idx)
    sampling_step!(mc_params,mc_states,ensemble,idx,results)
    #     if save == true
#         if rem(i,1000) == 0
#             save_states(mc_params,mc_states,i,save_dir)
#             save_results(results,save_dir)
#         end
#     end


    return mc_states 
end
    

In [ ]:
function equilibration_cycle!(mc_states,move_strat,mc_params,pot,ensemble,n_steps,results)
    #set initial hamiltonian values and ebounds
    for state in mc_states
        push!(mc_state.ham, 0)
        push!(mc_state.ham, 0)
    end
    ebounds = [100. , -100.]
    #begin equilibration
    for i = 1:mc_params.eq_cycles
        mc_states = mc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,n_steps,i)
        for state in mc_states
            ebounds = check_e_bounds(state.en_tot,ebounds)
        end
    end
    #post equilibration reset
    for state in mc_states
        reset_counters(state)
    end
    results = initialise_histograms!(mc_params,results,ebounds,mc_states[1].config.bc)
    return mc_states,results
end



In [ ]:
function ptmc_run!(mc_params::MCParams,temps::TempGrid,start_config::Config,potential::Ptype,ensemble::Etype;restart=false min_acc=0.4,max_acc=0.6,save=false,save_dir=pwd()) where Ptype <: AbstractPotential where Etype <: AbstractEnsemble
    #initialise the states and results etc
    mc_states,move_strategy,results,start_counter,n_steps = initialisation(mc_params,temp,start_config,potential,ensemble)
    println("params set")
    #Equilibration 
    mc_states,results = equilibration(mc_states,move_strat,mc_params,potential,ensemble,n_steps,results,restart)
    println("equilibration complete")

    #main loop 
    for i = start_counter:mc_params.mc_cycles 
        @inbounds mc_cycle!(mc_states,move_strategy,mc_params,potential,ensemble,n_steps,results,i)
    end
    println("MC loop done.")

    #Finalisation of results
    results = finalise_results(mc_states,mc_params,results)
    println("done")
    return mc_states,results
end

